Author: Yu Bai, Yutian Chen, Tianxiao Wei

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
np.set_printoptions(precision=2)

In [3]:
delta_t = 0.001
pi = np.pi

def pwm2rot(pwm):
        # maps from PWM to rotational speed in rad/s
        return pwm * 100 * 2 * pi / 60

In [4]:
class envir:
    # the environment is a rectangular consisting 4 walls bounding an open space
    # the positions of the walls are x=0, y=0, x=length, y=width
    def __init__(self, length, width):
        self.length = length
        self.width = width

In [5]:
class robot:

    def __init__(self, envir, width, diameter, x0 = 0, y0 = 0, theta0 = 0, ifOutput = True, ifPlot = True):
        self.envir = envir
        self.width = width
        self.diameter = diameter
        self.x = x0
        self.y = y0
        self.theta = theta0
        self.front_dist = 0
        self.right_dist = 0
        self.rot_speed = 0
        self.ns_mag_field = 0
        self.ew_mag_field = 0
        self.update_sensor()
        self.measure()
        if ifOutput:
            self.output()
        if ifPlot:
            self.plot()

    def move(self, pwm_l, pwm_r, ifOutput = True, ifPlot = True):
        width = self.width
        theta = self.theta
        # map from PWM to rotation speed
        rot_l = pwm2rot(pwm_l)
        rot_r = pwm2rot(pwm_r)

        # B is the matrix in x(t+1) = A*x(t) + B*u(t)
        B = delta_t * self.diameter/2 * np.array([[1/2 * np.cos(theta), 1/2 * np.cos(theta)],[1/2 * np.sin(theta), 1/2 * np.sin(theta)], [-1/width, 1/width]])
        # input is the input vector u(t)
        input = np.array([[rot_l],[rot_r]])

        # if the potential destination is out of the envrionment (blocked by a wall), it will not move
        [[pot_x], [pot_y], [pot_theta]] = [[self.x], [self.y], [self.theta]] + B.dot(input)
        # if pot_x < 0 or pot_x > self.envir.length or pot_y < 0 or pot_y > self.envir.width:
        #     # print("The robot didn't move because it is out of bound")
        #     if ifOutput:
        #         self.output()
        #     if ifPlot:
        #         self.plot()
        #     return

        # updates the system state with x(t+1) = A*x(t) + B*u(t)
        [self.x, self.y, self.theta] = [pot_x, pot_y, pot_theta]

        # reduce theta to be in the range of [-pi,pi)
        self.theta = np.remainder(self.theta, 2*pi)
        self.update_sensor() # update the sensors
        # updates the rotational speed
        self.rot_speed = self.diameter/(2 * width) * (rot_r - rot_l)
        self.measure() # provide sensor outputs
        if ifOutput:
            self.output()
        if ifPlot:
            self.plot()

    def update_sensor(self):
        # update the equations for the sensor lasers
        # y = mf*x + bf describes the laser from the front sensor
        # note that np.tan() will never be undefined due to the limitations of computer calculation,
        # thus we will have always have the function y = mf*x + bf for all states
        self.mf = np.tan(self.theta)
        self.bf = self.y - self.mf * self.x
        # y = mr*x + br describes the laser from the right sensor
        self.mr = np.tan(self.theta - pi/2)
        self.br = self.y - self.mr * self.x

    def measure(self):
        # in simulation, instead of directly obtaining outputs from the sensors, we calculate the sensor outputs with current state, input, and noise
        # calculate the intersection and distance between the front laser and the rectangular walls
        # if we know the x-coordinate of the wall, y = mx + b
        # if we know the y-coordinate of the wall, x = (y-b)/m
        l = self.envir.length
        w = self.envir.width
        mf = self.mf
        bf = self.bf
        mr = self.mr
        br = self.br

        # front_inter[0] to front_inter[4] are the intersections with the walls on the right, top, left, buttom, right
        # front_inter[4] is used to simplify calculations
        front_inter = np.zeros((5,2))
        if l*mf+bf >=0 and l*mf+bf <= w:
            front_inter[0] = [l, l*mf+bf]
            front_inter[4] = [l, l*mf+bf]
        if (w-bf)/mf >= 0 and (w-bf)/mf <= l: 
            front_inter[1] = [(w-bf)/mf, w]
        if bf >=0 and bf <= w:
            front_inter[2] = [0, bf]
        if -bf/mf >= 0 and -bf/mf <= l:
            front_inter[3] = [-bf/mf, 0]

        # if 0 <= theta < pi/2, we only need to consider the intersection with the right or top wall
        # if pi/2 <= theta < pi, we only need to consider the intersection with the top or left wall
        # if pi <= theta < 3pi/2, we only need to consider the intersection with the left or buttom wall
        # if 3pi/2 <= theta < 2pi, we only need to consider the intersection with the buttom or right wall
        # therefore, we use np.floor(theta/(pi/2)) to determine which walls need to be considered
        # since the laser y=mx+b will only intersect with one of the two adajacent walls, and the coordinate is set to [0,0] if there is no intersection,
        # we can directly compute the intersection by adding the values of adjacent rows in front_inter
        idx = int(np.floor(self.theta/(pi/2)))
        if idx == 4: # this could happen with rounding errors
            idx = 0
        front_inter_loc = front_inter[idx] + front_inter[idx+1]
        self.front_dist = np.sqrt(np.power(self.x-front_inter_loc[0],2) + np.power(self.y-front_inter_loc[1],2))

        # right_inter[0] to right_inter[4] are the intersections with the walls on the buttom, right, top, left, buttom
        # right_inter[4] is used to simply future calculations  
        right_inter = np.zeros((5,2))
        if -br/mr >= 0 and -br/mr <= l:
            right_inter[0] = [-br/mr, 0]
            right_inter[4] = [-br/mr, 0]
        if l*mr+br >=0 and l*mr+br <= w:
            right_inter[1] = [l, l*mr+br]
        if (w-br)/mr >= 0 and (w-br)/mr <= l: 
            right_inter[2] = [(w-br)/mr, w]
        if br >=0 and br <= w:
            right_inter[3] = [0, br]

        # similar to the calculation of the front intersection
        idx = int(np.floor(self.theta/(pi/2)))
        if idx == 4: # this could happen with rounding errors
            idx = 0
        right_inter_loc = right_inter[idx] + right_inter[idx+1]
        self.right_dist = np.sqrt(np.power(self.x-right_inter_loc[0],2) + np.power(self.y-right_inter_loc[1],2))


        # calculate the north-south and east-west magnetic field
        self.ns_mag_field = np.sin(self.theta)
        self.ew_mag_field = np.cos(self.theta)


    def output(self):
        print("x, y, theta                = ", ', '.join(str(i) for i in [self.x, self.y, self.theta]))
        print("front distance             = ", self.front_dist)
        print("right distance             = ", self.right_dist)
        print("rotational speed           = ", self.rot_speed)
        print("north-south magnetic field = ", self.ns_mag_field)
        print("east-west magnetic field   = ", self.ew_mag_field)

    def plot(self):
        plt.figure(figsize = (10,10))
        # plot the rectangular environment
        plt.vlines(x=[0, self.envir.length], ymin=0, ymax=self.envir.width)
        plt.hlines(y=[0, self.envir.width], xmin=0, xmax=self.envir.length)
        # plot the position of the robot
        plt.plot(self.x, self.y, 'ro')
        # plot the front and right sensor
        plt.quiver([self.x, self.x], [self.y, self.y],[np.cos(self.theta), np.sin(self.theta)], [np.cos(self.theta-pi/2), np.sin(self.theta-pi/2)], 
                   angles='xy', scale_units='xy', scale=1)
        plt.show()

## Joint Lab 3


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


get_control_signals returns a n*2 matrix of the inputs to the simulation system


In [7]:
def get_control_signals(trajectory_num):
    data = pd.read_csv('/content/drive/My Drive/Joint Lab 3 Data/trajectory.csv',usecols=['L'+str(trajectory_num),'R'+str(trajectory_num)])
    data = data.to_numpy()/600
    control_signals = np.zeros((10*1000,2))
    for i in range(20):
        v_init = data[i]
        v_end = data[i+1]
        for j in range(500):
            v = v_init + (v_end - v_init)*j/500
            control_signals[i*500+j] = v
    return control_signals

SEGWAY \\
This block compares simulations for the first 16 trajectories. \\
Plots and a table of cosine similarities are generated


In [ ]:
cos_sim = np.zeros((16,8))
for j in range(16):
    trajectory_num = j+1
    e = envir(10,10)
    segway = robot(e,width=0.53,diameter=0.502,x0=5,y0=5,theta0=0,ifOutput=False,ifPlot=False) # initial conditions for robots
    data_py = np.zeros((8,10000))

    # run the python simulation and records data
    control_signals = get_control_signals(trajectory_num)
    for i in range(10000):
        v_l = control_signals[i][0]
        v_r = control_signals[i][1]
        segway.move(v_l,v_r,ifOutput=False,ifPlot=False)
        data_py[0][i] = segway.x-5
        data_py[1][i] = segway.y-5
        data_py[2][i] = segway.theta
        data_py[3][i] = segway.front_dist
        data_py[4][i] = segway.right_dist
        data_py[5][i] = segway.rot_speed
        data_py[6][i] = segway.ns_mag_field
        data_py[7][i] = segway.ew_mag_field

    # theta_py = theta_py * (theta_py <= pi) + (theta_py-2*pi) * (theta_py > pi)
    data_py[2] = data_py[2] * (data_py[2] <= pi) + (data_py[2]-2*pi) * (data_py[2] > pi)

    data_wb = np.zeros((8,10000))
    # read data from webot simulation
    # x,y,theta,front distance,right distance,rotational speed,ns magnetic field,ew magnetic field
    with open('/content/drive/My Drive/Joint Lab 3 Data/data' + str(trajectory_num) + '.csv', 'r') as file:
        reader = csv.reader(file)
        for i, row in enumerate(reader):
            data_wb[i] = row

    cos_sim[j] = np.sum(data_py*data_wb,axis=1) / (np.linalg.norm(data_py,axis=1) * np.linalg.norm(data_wb,axis=1))

    # if trajectory_num in [13,14,15,16]:
    plt.plot(data_py[0],data_py[1])
    plt.title('Python Simulation for Trajectory ' + str(trajectory_num))
    plt.savefig('/content/drive/My Drive/Joint Lab 3 Data/plots/segway_py_' + str(trajectory_num))
    plt.show()
    plt.plot(data_wb[0],data_wb[1])
    plt.title('Webot Simulation for Trajectory ' + str(trajectory_num))
    plt.savefig('/content/drive/My Drive/Joint Lab 3 Data/plots/segway_wb_' + str(trajectory_num))
    plt.show()


print(cos_sim)
np.savetxt('/content/drive/My Drive/Joint Lab 3 Data/output.csv', cos_sim, delimiter=",")

This block compares simulations, from trajectory 17, 18 and 19, that have different (divergent path) behaviors \\
1. moves in a straight line and hit the wall (environment) \\
2. the acceleration of the angular velocities of the wheels is too high (accelerating too fast) \\
3. the torque applied to the robot is too high (making sharp turns)

In [ ]:
cos_sim = np.zeros((16,8))
for trajectory_num in [17,18,19]:
    e = envir(10,10)
    segway = robot(e,width=0.53,diameter=0.502,x0=5,y0=5,theta0=0,ifOutput=False,ifPlot=False) # initial conditions for robots
    data_py = np.zeros((8,10000))

    # run the python simulation and records data
    control_signals = get_control_signals(trajectory_num)
    for i in range(10000):
        v_l = control_signals[i][0]
        v_r = control_signals[i][1]
        segway.move(v_l,v_r,ifOutput=False,ifPlot=False)
        data_py[0][i] = segway.x-5
        data_py[1][i] = segway.y-5
        data_py[2][i] = segway.theta
        data_py[3][i] = segway.front_dist
        data_py[4][i] = segway.right_dist
        data_py[5][i] = segway.rot_speed
        data_py[6][i] = segway.ns_mag_field
        data_py[7][i] = segway.ew_mag_field

    # theta_py = theta_py * (theta_py <= pi) + (theta_py-2*pi) * (theta_py > pi)
    data_py[2] = data_py[2] * (data_py[2] <= pi) + (data_py[2]-2*pi) * (data_py[2] > pi)

    data_wb = np.zeros((8,10000))
    # read data from webot simulation
    # x,y,theta,front distance,right distance,rotational speed,ns magnetic field,ew magnetic field
    with open('/content/drive/My Drive/Joint Lab 3 Data/data' + str(trajectory_num) + '.csv', 'r') as file:
        reader = csv.reader(file)
        for i, row in enumerate(reader):
            data_wb[i] = row

    cos_sim[trajectory_num-17] = np.sum(data_py*data_wb,axis=1) / (np.linalg.norm(data_py,axis=1) * np.linalg.norm(data_wb,axis=1))

    plt.plot(data_py[0],data_py[1])
    plt.title('Python Simulation for Trajectory ' + str(trajectory_num))
    plt.savefig('/content/drive/My Drive/Joint Lab 3 Data/plots/*segway_py_' + str(trajectory_num))
    plt.show()
    plt.plot(data_wb[0],data_wb[1])
    plt.title('Webot Simulation for Trajectory ' + str(trajectory_num))
    plt.savefig('/content/drive/My Drive/Joint Lab 3 Data/plots/*segway_wb_' + str(trajectory_num))
    plt.show()

PAPERBOT

In [ ]:
cos_sim = np.zeros((3,8))
diff = np.zeros((3,8))
for j in range(3):
    trajectory_num = j+1
    e = envir(1,1)
    paperbot = robot(e,width=0.09,diameter=0.050,x0=5,y0=5,theta0=0,ifOutput=False,ifPlot=False) # initial conditions for robots
    data_py = np.zeros((8,10000))

    # run the python simulation and records data
    control_signals = get_control_signals(trajectory_num)
    for i in range(10000):
        v_l = control_signals[i][0]
        v_r = control_signals[i][1]
        paperbot.move(v_l,v_r,ifOutput=False,ifPlot=False)
        data_py[0][i] = paperbot.x-5
        data_py[1][i] = paperbot.y-5
        data_py[2][i] = paperbot.theta
        data_py[3][i] = paperbot.front_dist
        data_py[4][i] = paperbot.right_dist
        data_py[5][i] = paperbot.rot_speed
        data_py[6][i] = paperbot.ns_mag_field
        data_py[7][i] = paperbot.ew_mag_field

    # theta_py = theta_py * (theta_py <= pi) + (theta_py-2*pi) * (theta_py > pi)
    data_py[2] = data_py[2] * (data_py[2] <= pi) + (data_py[2]-2*pi) * (data_py[2] > pi)

    data_wb = np.zeros((8,10000))
    # read data from webot simulation
    # x,y,theta,front distance,right distance,rotational speed,ns magnetic field,ew magnetic field
    with open('/content/drive/My Drive/ECE183DA/Joint Lab 3 Data/data' + str(trajectory_num) + '.csv', 'r') as file:
        reader = csv.reader(file)
        for i, row in enumerate(reader):
            data_wb[i] = row

    cos_sim[j] = np.sum(data_py*data_wb,axis=1) / (np.linalg.norm(data_py,axis=1) * np.linalg.norm(data_wb,axis=1))

    # if trajectory_num in [13,14,15,16]:
    plt.plot(data_py[0],data_py[1])
    plt.title('Python Simulation for Trajectory ' + str(trajectory_num))
    plt.savefig('/content/drive/My Drive/ECE183DA/Joint Lab 3 Data/plots/paperbot_py_' + str(trajectory_num))
    plt.show()
    plt.plot(data_wb[0],data_wb[1])
    plt.title('Webots Simulation for Trajectory ' + str(trajectory_num))
    plt.savefig('/content/drive/My Drive/ECE183DA/Joint Lab 3 Data/plots/paperbot_wb_' + str(trajectory_num))
    plt.show()


print(cos_sim)
np.savetxt('/content/drive/My Drive/ECE183DA/Joint Lab 3 Data/output.csv', cos_sim, delimiter=",")

In [ ]:
test = [1]
for j in test:
    trajectory_num = j+1
    e = envir(10,10)
    paperbot = robot(e,width=0.53,diameter=0.502,x0=5,y0=5,theta0=0,ifOutput=False,ifPlot=False) # initial conditions for robots
    data_py = np.zeros((8,10000))

    # run the python simulation and records data
    control_signals = get_control_signals(trajectory_num)
    for i in range(10000):
        v_l = control_signals[i][0]
        v_r = control_signals[i][1]
        paperbot.move(v_l,v_r,ifOutput=False,ifPlot=False)
        data_py[0][i] = paperbot.x-5
        data_py[1][i] = paperbot.y-5
        data_py[2][i] = paperbot.theta
        data_py[3][i] = paperbot.front_dist
        data_py[4][i] = paperbot.right_dist
        data_py[5][i] = paperbot.rot_speed
        data_py[6][i] = paperbot.ns_mag_field
        data_py[7][i] = paperbot.ew_mag_field

    # theta_py = theta_py * (theta_py <= pi) + (theta_py-2*pi) * (theta_py > pi)
    data_py[2] = data_py[2] * (data_py[2] <= pi) + (data_py[2]-2*pi) * (data_py[2] > pi)

    data_wb = np.zeros((8,10000))
    # read data from webot simulation
    # x,y,theta,front distance,right distance,rotational speed,ns magnetic field,ew magnetic field
    with open('/content/drive/My Drive/ECE183DA/Joint Lab 3 Data/data' + str(trajectory_num) + '.csv', 'r') as file:
        reader = csv.reader(file)
        for i, row in enumerate(reader):
            data_wb[i] = row
    
    cos_sim = np.sum(data_py*data_wb,axis=1) / (np.linalg.norm(data_py,axis=1) * np.linalg.norm(data_wb,axis=1))
    print(cos_sim)

    # if trajectory_num in [13,14,15,16]:
    plt.plot(data_py[0],data_py[1])
    plt.title('Python Simulation for Trajectory ' + str(trajectory_num))
    plt.show()
    plt.plot(data_wb[0],data_wb[1])
    plt.title('Webots Simulation for Trajectory ' + str(trajectory_num))
    plt.show()